In [ ]:
import os
import subprocess
git_root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).decode().strip()
%load_ext autoreload
%autoreload 2
os.chdir(git_root)
import ultralytics
print(os.path.dirname(ultralytics.__file__))
# https://docs.ultralytics.com/tasks/obb/#__tabbed_1_1
# https://docs.ultralytics.com/datasets/obb/ OBB Format
from ultralytics import YOLO
# Load a model
model = YOLO('yolov8n-obb.yaml')  # build a new model from YAML
model = YOLO('yolov8n-obb.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8n-obb.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data='DOTAv2.0-patches.yaml', epochs=20, imgsz=640, fraction=.01, batch=5)	 # data='dota8.yaml' multi_scale=True,mixup=1.0, 


In [ ]:
%%bash
# This script takes v1.0 and v2.0 original datasets and merges into ./examples/datasets/DOTA-v2.0 (DOTA format)
DATA_DIR=/home/vscode/datasets
echo $PWD
# Remove all previously exported files if they exist.
rm -rf $DATA_DIR/dota && mkdir -p $DATA_DIR/dota
# Remove DOTA-v2.0 dataset if exists.
rm -rf ./examples/datasets/DOTA-v2.0/ && mkdir -p ./examples/datasets/DOTA-v2.0/

rm -rf  $DATA_DIR/dota/v1.0 $DATA_DIR/dota/v2.0 && mkdir -p $DATA_DIR/dota/v1.0 $DATA_DIR/dota/v2.0
unzip -o -q -d $DATA_DIR/dota/v1.0 $DATA_DIR/dota-v1.0.zip
unzip -o -q -d $DATA_DIR/dota/v2.0 $DATA_DIR/dota-v2.0.zip
# Create directory structure for DOTA-v2.0 dataset according to ultralytics/yolov8
mkdir -p ./examples/datasets/DOTA-v2.0/{images,labels}/{train,val}

# Move files from v1.0 validation set to v2.0 validation set
for split in val train; do
    # Move data from v1.0,v2.0 to DOTA-v2.0/images
    find $DATA_DIR/dota/{v1.0,v2.0}/$split/images -name '*.png' -exec mv {} ./examples/datasets/DOTA-v2.0/images/$split \;
    # Move labels only from v2.0 to DOTA-v2.0/labels
    find $DATA_DIR/dota/v2.0/$split/labels -name '*.txt' -exec mv {} ./examples/datasets/DOTA-v2.0/labels/$split \;
done

In [ ]:
# Takes the original labels and converts them from DOTA to YOLO format

# Need to move them to an original directory to prepare for DOTA->YOLO conversion
!mv ./examples/datasets/DOTA-v2.0/labels/train ./examples/datasets/DOTA-v2.0/labels/train_original 
!mv ./examples/datasets/DOTA-v2.0/labels/val ./examples/datasets/DOTA-v2.0/labels/val_original
# Then the DOTA format needs to be converted to YOLO-OBB
from ultralytics.data.converter import convert_dota_to_yolo_obb
convert_dota_to_yolo_obb('./examples/datasets/DOTA-v2.0')


| Data Format                | Description                                                                 | New Column                          |
|---------------------------|-----------------------------------------------------------------------------|-------------------------------------|
| DOTA                      | `x1, y1, x2, y2, x3, y3, x4, y4, category, difficult`                        | OG DOTA format                      |
| YOLO-OBB                  | `class_index, x1, y1, x2, y2, x3, y3, x4, y4`                                | YOLO-OBB expected input format      |
| YOLO-OBB-internal         | `x,y,w,h, r`                                                               | YOLO-OBB internal format             |

### Data formatting in DOTA

There are three data formats:

- An image corresponds to a text file with the annotations i.e, **N** samples : `.png <--> .txt`

- Convert DOTA to `YOLO-OBB` using         

```python
from ultralytics.data.converter import convert_dota_to_yolo_obb

convert_dota_to_yolo_obb('path/to/DOTA')
```


### Baselines

Visit [here](https://github.com/dingjiansw101/AerialDetection/blob/master/MODEL_ZOO.md) for baselines

In [ ]:
# Creates a split where smalled images are cropped from large ones.

! rm -rf ./examples/datasets/DOTA-v2.0-split
from ultralytics.data.split_dota import split_trainval, split_test
from PIL import Image
Image.MAX_IMAGE_PIXELS = None # There is a security issue related to this.
# split train and val set, with labels.
split_trainval(
    data_root='./examples/datasets/DOTA-v2.0',
    save_dir='./examples/datasets/DOTA-v2.0-patches',
    rates=[0.5, 1.0, 1.5],    # multi-scale
    gap=500
)


In [ ]:
# NO test set right now.
# split test set, without labels.
if False:
    split_test(
        data_root='./examples/datasets/DOTA-v2.0',
        save_dir='./examples/datasets/DOTA-v2.0-split',
        rates=[0.5, 1.0, 1.5],    # multi-scale
        gap=500
    )

### Dev-env notes
- Remember pip install wandb
- Install ultralytics with `pip -e .`
- Create `./scripts/.env` file, put the wandb secret there
- If you dont want to log to wandb remember env `WANDB_MODE=offline`
- Remember to set `datasets_dirs: (git repo root)/examples/datasets` in ultralytics config at `~/.config/Ultralytics/settings.yaml`


### General Notes

- Read A.Karpathy notes on training NNs [here](https://karpathy.github.io/2019/04/25/recipe/)
- Note on missing labels and how important they are [here](https://docs.ultralytics.com/yolov5/tutorials/tips_for_best_training_results/)

### Results

![PR Curve](../runs/obb/val5/PR_curve.png)
![Image](../runs/obb/val5/val_batch0_pred.jpg)
